# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [44]:
# Import dependencies
!pip install pymongo dnspython
from pymongo import MongoClient
from pprint import pprint

In [45]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [46]:
# confirm that our new database was created
db_list = mongo.list_database_names()
print("Databases:", db_list)

Databases: ['admin', 'config', 'local', 'uk_food']


In [47]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [48]:
# review the collections in our new database
db_list = mongo.list_database_names()
print("Databases:", db_list)

Databases: ['admin', 'config', 'local', 'uk_food']


In [71]:
# review the collections in our new database
collection = db['establishments']
print("Databases:",collection)

Databases: Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')


In [72]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [73]:
# Create a dictionary for the new restaurant data
new_data = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [75]:
# Insert the new restaurant into the collection
establishments.insert_one(new_data)

InsertOneResult(ObjectId('65ca98eaccf836a2e9457c0f'), acknowledged=True)

In [76]:
# Check that the new restaurant was inserted
new_restaurant_name = "Penang Flavours"

# Perform a search query
result = establishments.find_one({"BusinessName": new_restaurant_name})

# Check the result
if result:
    print("New restaurant was successfully inserted:", result)
else:
    print("Restaurant not found.")

New restaurant was successfully inserted: {'_id': ObjectId('65ca98eaccf836a2e9457c0f'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [77]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

# Perform the query
result = establishments.find_one(query, projection)

# Check the result and print
if result:
    print("Query result:", result)
else:
    print("No document matches the specified criteria.")

Query result: {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [79]:
# Update the new restaurant with the correct BusinessTypeID
found_business_type_id = "1"

# Update criteria - identifying the document to update
update_criteria = {
    "BusinessName": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd"
}

# Update operation - setting the new BusinessTypeID
update_operation = {
    "$set": {"BusinessTypeID": found_business_type_id}
}

# Perform the update
result = establishments.update_one(update_criteria, update_operation)

# Check the result
if result.matched_count:
    print("The restaurant was successfully updated.")
else:
    print("No matching document found to update.")

The restaurant was successfully updated.


In [80]:
# Confirm that the new restaurant was updated
new_restaurant_name = "Penang Flavours"

# Perform a search query
result = establishments.find_one({"BusinessName": new_restaurant_name})

# Check the result
if result:
    print("New restaurant was successfully inserted:", result)
else:
    print("Restaurant not found.")

New restaurant was successfully inserted: {'_id': ObjectId('65ca98eaccf836a2e9457c0f'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '1', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [84]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Documents with LocalAuthorityName as 'Dover': {dover_count}")

Documents with LocalAuthorityName as 'Dover': 994


In [85]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})

# Printing the number of documents deleted
print(f"Documents deleted: {delete_result.deleted_count}")

Documents deleted: 994


In [86]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Documents with LocalAuthorityName as 'Dover': {dover_count}")

Documents with LocalAuthorityName as 'Dover': 0


In [88]:
# Check that other documents remain with 'find_one'
dover_document = establishments.find_one({"LocalAuthorityName": "Dover"})

# Check if a document was found
if dover_document:
    print("A document with LocalAuthorityName 'Dover' exists in the collection:", dover_document)
else:
    print("No documents with LocalAuthorityName 'Dover' found in the collection.")

No documents with LocalAuthorityName 'Dover' found in the collection.


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [89]:
# Change the data type from String to Decimal for longitude and latitude
pipeline = [
    {
        '$set': {
            'geocode.longitude': {
                '$toDecimal': '$geocode.longitude'
            },
            'geocode.latitude': {
                '$toDecimal': '$geocode.latitude'
            }
        }
    }
]

# Execute the update_many operation with an empty filter to update all documents
establishments.update_many({}, pipeline)

print("Longitude and latitude data types updated to Decimal.")

Longitude and latitude data types updated to Decimal.


Use `update_many` to convert `RatingValue` to integer numbers.

In [90]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [91]:
# Change the data type from String to Integer for RatingValue
pipeline = [
    {
        '$set': {
            'RatingValue': {
                '$toInt': '$RatingValue'  # Convert RatingValue to integer
            }
        }
    }
]


result = establishments.update_many({}, pipeline)

# Print the result of the operation
print(f"Documents modified: {result.modified_count}")

Documents modified: 34695


In [92]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()

# Check and print the data types of the fields
if sample_document:
    print("Sample Document:")
    print(sample_document)
    print("\nData Types:")
    print(f"Longitude Type: {type(sample_document['geocode']['longitude'])}")
    print(f"Latitude Type: {type(sample_document['geocode']['latitude'])}")
    print(f"RatingValue Type: {type(sample_document['RatingValue'])}")
else:
    print("No document found.")

Sample Document:
{'_id': ObjectId('65c74a5322afc552b0a57718'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('1.19640800000000'), 'latitude': Decimal128('51.0860580000000')}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': 